#Морфологический анализ текста на русском языке
##pymorphy2
Pymorphy2 — морфологический процессор с открытым исходным
кодом, предоставляет все функции полного морфологического анализа и
синтеза словоформ. Процессор базируется на словарной морфологии и использует словарные данные проекта **OpenCorpora**


Установка

In [2]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 77.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13707 sha256=8fe0fe70ed050e82de4f3ea0e70c669ad703af559fb57fa7f9e73af19e23a19d
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


Словари распространяются отдельными пакетами и требуют периодически обновлений

In [3]:
!pip install -U pymorphy2-dicts-ru

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import pymorphy2

В pymorphy2 для морфологического анализа слов есть класс ``` MorphAnalyzer ``` (по умолчанию стоит русский язык)

In [5]:
morph = pymorphy2.MorphAnalyzer()

С помощью метода `MorphAnalyzer.parse()` можно разобрать отдельное слово. Метод возвращает один или несколько объектов типа `Parse` с информацией о том, как слово может быть разобрано. Ниже проведен разбор слова 'стали'.

In [6]:
morph.parse('стали')

[Parse(word='стали', tag=OpencorporaTag('VERB,perf,intr plur,past,indc'), normal_form='стать', score=0.975342, methods_stack=((DictionaryAnalyzer(), 'стали', 945, 4),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn sing,gent'), normal_form='сталь', score=0.010958, methods_stack=((DictionaryAnalyzer(), 'стали', 13, 1),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn plur,nomn'), normal_form='сталь', score=0.005479, methods_stack=((DictionaryAnalyzer(), 'стали', 13, 6),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn sing,datv'), normal_form='сталь', score=0.002739, methods_stack=((DictionaryAnalyzer(), 'стали', 13, 2),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn sing,loct'), normal_form='сталь', score=0.002739, methods_stack=((DictionaryAnalyzer(), 'стали', 13, 5),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn plur,accs'), normal_form='сталь', score=0.002739, methods_stack=((DictionaryAnalyzer(), 'стали', 13, 9),))]

Структура ответа состоит из поля:
- word – исходное слово;
- tag – грамматические характеристики; 
  *   например **OpencorporaTag('VERB,perf,intr plur,past,indc')** дает следующую информацию: слово - глагол (VERB) совершенного вида (perf), непереходный (intr), множественного числа (plur), прошедшего времени (past), изъявительного наклонения (indc). [Обозначения для граммем.](https://pymorphy2.readthedocs.io/en/stable/user/grammemes.html#grammeme-docs)
- normal_form – начальная форма слова; 
- score – это оценка $P(tag|word)$ вероятности того, что данный разбор правильный.


###Выбор правильного разбора
pymorphy2 возвращает все допустимые варианты разбора, но на практике обычно нужен только один вариант, правильный. Для этого у разбора есть параметр score. Условная вероятность $P(tag|word)$ оценивается на основе корпуса **OpenCorpora**: ищутся все неоднозначные слова со снятой неоднозначностью, для каждого слова считается, сколько раз ему был сопоставлен данный тег, и на основе этих частот вычисляется условная вероятность тега (с использованием сглаживания Лапласа): 
$$P(tag|word) = \dfrac{Fr(tag, word) + 1}{Fr(word) + R(word)} $$
где $Fr(tag, word)$ - количество раз, когда данная словоформа $word$ встретилась с тегом (т.е. с данными грамматическими характеристиками) $tag$ в корпусе **OpenCorpora**, $Fr(word)$ - количество раз, когда встретилась данная словоформа (уже без учета тега), $R(word)$ - число выведенных разборов анализатора для нашего слова $word$.

Разборы сортируются по убыванию score, поэтому первый разбор `morph.parse('стали')[0]` наиболее вероятный. 

Оценки $P(tag|word)$ помогают улучшить разбор, но их недостаточно для надежного снятия неоднозначности, как минимум по следующим причинам:
*   то, как нужно разбирать слово, зависит от соседних слов, а **pymorphy2** работает только на уровне отдельных слов;
*   условная вероятность $P(tag|word)$ оценена на основе сбалансированного набора текстов; в специализированных текстах вероятности могут быть другими - например, возможно, что в металлургических текстах $P(NOUN|стали) \gt P(VERB|стали)$;
*   в OpenCorpora у большинства слов неоднозначность пока не снята; 

 ### Разбор несловарных слов


Дописать

In [7]:
morph.parse('т.д.')

[Parse(word='т.д.', tag=OpencorporaTag('UNKN'), normal_form='т.д.', score=1.0, methods_stack=((UnknAnalyzer(), 'т.д.'),))]

In [8]:
# inputFile = open('input.txt', 'r', 'cp1251')
# words = inputFile.readline()
# print(words)

### Предобработка текста
Для наилучшей оценки качества лемматизации с помощью pymorphy2 следует выполнить некоторую предобработку текста. В частности, необходимо:

* Очистить текст от знаков препинания, чисел и других символов, которые не являются словами.
* Привести все символы к нижнему регистру, чтобы избежать различий в написании одних и тех же слов в разных формах (например, "Стул" и "стул").
* Разбить текст на отдельные слова (токенизация), чтобы передать их pymorphy2 на лемматизацию.

Также стоит учитывать особенности конкретного языка, на котором написан текст, так как `pymorphy2` работает на основе словарей для каждого языка. Например, для русского языка стоит убедиться, что в тексте использованы правильные буквы ё и й.

In [9]:
import nltk
nltk.download('punkt')
nltk.download('stopwords') 

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [10]:
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [11]:
punctuation = punctuation + '—«»±§<>'

In [75]:
def preprocess_text(text):
    text = text.lower()  #нижний регистр
    text = ''.join([char for char in text if char not in punctuation and not char.isdigit()]) # удаление знаков препинания и цифр

    tokens = word_tokenize(text) #токенизация
    
    stop_words = set(stopwords.words('russian'))  # стоп-слова (предлоги, местоимения и пр.)
    tokens = [token for token in tokens if token not in stop_words]
    
    return tokens

Чуть подробнее рассмотрим момент с удалением стоп-слов. 

Стоп-слова - это слова, которые встречаются в тексте очень часто, но не несут особой смысловой нагрузки. К ним относятся местоимения, предлоги, союзы и т.д.

Местоимения являются стоп-словами, потому что они заменяют существительные, прилагательные и другие части речи, которые обычно не являются стоп-словами. Например, слово "он" может заменять существительные "человек", "мужчина", "парень" и т.д., поэтому его употребление не несет большого значения для понимания содержания текста.

Важно отметить, что использование местоимений может быть полезным для избегания повторений в тексте и делает его более легким для чтения. Однако, при анализе текста или разработке алгоритмов обработки естественного языка, стоп-слова, включая местоимения, часто удаляются, чтобы упростить анализ и повысить точность результатов.

In [76]:
text = 'Оленька, дочь отставного коллежского асессора Племянникова, сидела у себя во дворе на крылечке, задумавшись. Было жарко, назойливо приставали мухи, и было так приятно думать, что скоро уже вечер. С востока надвигались темные дождевые тучи, и оттуда изредка потягивало влагой. Среди двора стоял Кукин, антрепренер и содержатель увеселительного сада «Тиволи», квартировавший тут же во дворе, во флигеле, и глядел на небо.— Опять! — говорил он с отчаянием. — Опять будет дождь! Каждый день дожди, каждый день дожди — точно нарочно! Ведь это петля! Это разоренье! Каждый день страшные убытки!Он всплеснул руками и продолжал, обращаясь к Оленьке:— Вот вам, Ольга Семеновна, наша жизнь. Хоть плачь! Работаешь, стараешься, мучишься, ночей не спишь, всё думаешь, как бы лучше, — и что же? С одной стороны, публика, невежественная, дикая. Даю ей самую лучшую оперетку, феерию, великолепных куплетистов, по разве ей это нужно? Разве она в этом понимает что-нибудь? Ей нужен балаган! Ей подавай пошлость! С другой стороны, взгляните на погоду. Почти каждый вечер дождь. Как зарядило с десятого мая, так потом весь май и июнь, просто ужас! Публика не ходит, но ведь я за аренду плачу? Артистам плачу?'

In [77]:
preprocess_text(text)

['оленька',
 'дочь',
 'отставного',
 'коллежского',
 'асессора',
 'племянникова',
 'сидела',
 'дворе',
 'крылечке',
 'задумавшись',
 'жарко',
 'назойливо',
 'приставали',
 'мухи',
 'приятно',
 'думать',
 'скоро',
 'вечер',
 'востока',
 'надвигались',
 'темные',
 'дождевые',
 'тучи',
 'оттуда',
 'изредка',
 'потягивало',
 'влагой',
 'среди',
 'двора',
 'стоял',
 'кукин',
 'антрепренер',
 'содержатель',
 'увеселительного',
 'сада',
 'тиволи',
 'квартировавший',
 'дворе',
 'флигеле',
 'глядел',
 'небо',
 'говорил',
 'отчаянием',
 'дождь',
 'каждый',
 'день',
 'дожди',
 'каждый',
 'день',
 'дожди',
 'точно',
 'нарочно',
 'это',
 'петля',
 'это',
 'разоренье',
 'каждый',
 'день',
 'страшные',
 'убыткион',
 'всплеснул',
 'руками',
 'продолжал',
 'обращаясь',
 'оленьке',
 'ольга',
 'семеновна',
 'наша',
 'жизнь',
 'плачь',
 'работаешь',
 'стараешься',
 'мучишься',
 'ночей',
 'спишь',
 'всё',
 'думаешь',
 'одной',
 'стороны',
 'публика',
 'невежественная',
 'дикая',
 'даю',
 'самую',
 'лучшую'

In [15]:
text1 = 'Меня зовут К.С.Астанова.'

In [16]:
preprocess_text(text1)

['зовут', 'ксастанова']

*Вопросы: насколько актуально заняться удалением названий городов, имен, аккуратнее обрабатывать случаи сокращений ('т.д.')*

### Лемматизация

После проведения предобработки текста можно использовать функцию лемматизации в pymorphy2 для получения базовых форм слов.

*Вопросы: следует ли испривать нормализацию "думающих" -> "думать" в "думающих" -> "думающий" или оставить так*

In [82]:
morph = pymorphy2.MorphAnalyzer()

def pymorphy_lemmatize(tokens):
    #res_line = []
    lemmas = [morph.parse(token)[0].normal_form for token in tokens]
    # for token in tokens:
    #     print(token)
    #     res_line.append(morph.parse(token)[0].normal_form)
    return lemmas

In [84]:
tokens = preprocess_text(text)
normalized = pymorphy_lemmatize(tokens)
print(' '.join(normalized))

оленька дочь отставной коллежский асессор племянников сидеть двор крылечко задуматься жарко назойливый приставать муха приятно думать скоро вечер восток надвигаться тёмный дождевой туча оттуда изредка потягивать влага среди двор стоять кукин антрепренёр содержатель увеселительный сад тиволя квартировать двор флигель глядеть небо говорить отчаяние дождь каждый день дождь каждый день дождь точно нарочно это петлить это разорение каждый день страшный убыткион всплеснуть рука продолжать обращаться оленька ольга семёнович наш жизнь плакать работать стараться мучиться ночь спать всё думать один сторона публика невежественный дикий давать самый хороший оперетка феерия великолепный куплетист это нужно понимать чтонибыть нужный балаган подавать пошлость сторона взглянуть погода каждый вечер дождь зарядить десятый май весь май июнь просто ужас публика ходить аренда плач артист плач


##pymystem3
Этот модуль содержит оболочку для морфологического анализатора русского языка Yandex Mystem 3.1, выпущенного в июне 2014 года. Морфологический анализатор может осуществлять лемматизацию текста и выводить набор морфологических атрибутов для каждого токена.

Реализован для русского, польского и английского языков.

In [50]:
!pip install pymystem3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


###Основные методы
`Mystem()` - конструктор класса, который инициализирует экземпляр анализатора Mystem.
Основными методами в `pymystem3` являются:
* `analyze(text)` - метод, который принимает на вход текст и возвращает список словарей, каждый из которых представляет отдельное слово с его леммой и грамматическими характеристиками.
* `lemmatize(text)` - метод, который принимает на вход текст и возвращает список лемм (основных форм слов).

Посмотрим примеры использования функционала. Проведем лемматизацию текста. `tokens` - список токенов из предыдущей ячейки после предобработки текста.

In [47]:
from pymystem3 import Mystem

mystem = Mystem()
lemmas = mystem.lemmatize(' '.join(tokens))

print("lemmas:", ''.join(lemmas))

lemmas: оленька дочь отставной коллежский асессор племянников сидеть двор крылечко задумываться жарко назойливый приставать муха приятно думать скоро вечер восток надвигаться темный дождевой туча оттуда изредка потягивать влага среди двор стоять кукин антрепренер содержатель увеселительный сад тиволя квартировать двор флигель глядеть небо говорить отчаяние дождь каждый день дождь каждый день дождь точно нарочно это петля этот разорение каждый день страшный убыткион всплескивать рука продолжать обращаться оленек ольга семеновна наш жизнь плакать работать стараться мучиться ночь спать все думать один сторона публика невежественный дикий давать самый хороший оперетка феерия великолепный куплетист это нужно понимать чтонибудь нужный балаган подавать пошлость сторона взглядывать погода каждый вечер дождь заряжать десятый май весь май июнь просто ужас публика ходить аренда плакать артист плакать



In [48]:
analysis = mystem.analyze(' '.join(tokens))
analysis

[{'analysis': [{'lex': 'оленька',
    'wt': 0.8709863567,
    'gr': 'S,имя,жен,од=им,ед'}],
  'text': 'оленька'},
 {'text': ' '},
 {'analysis': [{'lex': 'дочь', 'wt': 1, 'gr': 'S,жен,од=(вин,ед|им,ед)'}],
  'text': 'дочь'},
 {'text': ' '},
 {'analysis': [{'lex': 'отставной',
    'wt': 1,
    'gr': 'A=(вин,ед,полн,муж,од|род,ед,полн,муж|род,ед,полн,сред)'}],
  'text': 'отставного'},
 {'text': ' '},
 {'analysis': [{'lex': 'коллежский',
    'wt': 1,
    'gr': 'A=(вин,ед,полн,муж,од|род,ед,полн,муж|род,ед,полн,сред)'}],
  'text': 'коллежского'},
 {'text': ' '},
 {'analysis': [{'lex': 'асессор',
    'wt': 1,
    'gr': 'S,муж,од=(вин,ед|род,ед|им,мн)'}],
  'text': 'асессора'},
 {'text': ' '},
 {'analysis': [{'lex': 'племянников',
    'wt': 1,
    'gr': 'A,полн,притяж=(им,ед,жен|вин,ед,муж,од|род,ед,муж|род,ед,сред)'}],
  'text': 'племянникова'},
 {'text': ' '},
 {'analysis': [{'lex': 'сидеть',
    'wt': 1,
    'gr': 'V,несов,нп=прош,ед,изъяв,жен'}],
  'text': 'сидела'},
 {'text': ' '},
 {'an

##TreeTagger
TreeTagger - это программа лингвистического анализа текстов, разработанная командой исследователей в университете Штутгарта. Она используется для определения частей речи слов, извлечения лемм (основных форм слов) и выделения грамматических характеристик, таких как падеж, число и время.

Программа работает на основе статистических моделей, которые были обучены на большом количестве разнообразных текстов на разных языках. Для каждой языковой модели программа использует свой набор правил, которые позволяют повысить точность определения частей речи и других характеристик.

TreeTagger поддерживает множество языков, включая английский, немецкий, французский, испанский, итальянский, португальский, русский и другие.

In [51]:
!pip install treetaggerwrapper

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for treetaggerwrapper: filename=treetaggerwrapper-2.3-py3-none-any.whl size=40759 sha256=fc0e98722963cc8c131c3d1b276703e318094e343dc4eea611500d8534074530
  Stored in directory: /root/.cache/pip/wheels/ea/d5/4b/a29ceaa48c687208c69a791394c02c8e432971a98d8e5fc9ca
Successfully built treetaggerwrapper


In [52]:
import treetaggerwrapper

/usr/local/lib/python3.10/dist-packages/treetaggerwrapper.py:739: FutureWarning: Possible nested set at position 8
  punct2find_re = re.compile("([^ ])([[" + ALONEMARKS + "])",
/usr/local/lib/python3.10/dist-packages/treetaggerwrapper.py:2043: FutureWarning: Possible nested set at position 152
  DnsHostMatch_re = re.compile("(" + DnsHost_expression + ")",
/usr/local/lib/python3.10/dist-packages/treetaggerwrapper.py:2067: FutureWarning: Possible nested set at position 409
  UrlMatch_re = re.compile(UrlMatch_expression, re.VERBOSE | re.IGNORECASE)
/usr/local/lib/python3.10/dist-packages/treetaggerwrapper.py:2079: FutureWarning: Possible nested set at position 192
  EmailMatch_re = re.compile(EmailMatch_expression, re.VERBOSE | re.IGNORECASE)


In [54]:
def treetagger_lemmatize(text):
    tagger = treetaggerwrapper.TreeTagger(TAGLANG='ru')
    tags = tagger.tag_text(text)
    lemmas = []
    for tag in tags:
        parts = tag.split('\t')
        if len(parts) == 3:
            word, pos, lemma = parts
            lemmas.append(lemma)
    return lemmas

In [56]:
treetagger_lemmatize(text)

ERROR:TreeTagger:Failed to find TreeTagger from automatic directories list.
ERROR:TreeTagger:If you installed TreeTagger in a standard place, please contact the treetaggerwrapper author to add this place to this list.
ERROR:TreeTagger:To continue working, setup TAGDIR env var to TreeTagger directory.
ERROR:TreeTagger:Can't locate TreeTagger directory (and no TAGDIR specified).


TreeTaggerError: ignored

##Оценка производительности
Производительность морфологических анализаторов может оцениваться по нескольким критериям:

* Скорость работы - время, затрачиваемое на обработку одного слова или текста определенного объема.

* Точность - способность анализатора правильно определять лемму, часть речи и грамматические характеристики слова.

* Объем словаря - количество слов, включенных в словарь анализатора.

* Покрытие языка - доля всех возможных словоформ языка, которые могут быть обработаны анализатором.

* Надежность - степень уверенности анализатора в своих результатах при обработке текста с неожиданными словами или нетипичной грамматикой.

* Масштабируемость - способность анализатора работать с большими объемами данных без замедления производительности.

* Удобство использования - простота и интуитивный интерфейс для пользователя, возможность интеграции со сторонними программами и сервисами.

## Скорость работы


In [85]:
book = []
with open('./tihiy-don.txt', 'r', encoding='utf-8', newline='\n', errors='ignore') as resf:
    line = resf.readline()
    while line:
        tokens = preprocess_text(line)
        if len(tokens) > 0:
            book.append(tokens)
        line = resf.readline()

In [ ]:
book

In [89]:
from datetime import datetime

In [90]:
start_time = datetime.now()
res_book = []
for line in book:
    normalized = pymorphy_lemmatize(line)
    res_book.append(' '.join(normalized))

print('time = ', str(datetime.now() - start_time))

#open('./pymorphy2.txt', 'w', encoding='utf-8').writelines("\n".join(res_book))

time =  0:00:14.121161


In [95]:
length = 0
for line in res_book:
  length += len(line)
print('Result length: ',length)

Result length:  1041675


In [96]:
start_time = datetime.now()
res_book_mystem = []
for line in book:
    lemmas = mystem.lemmatize(' '.join(line))
    res_book_mystem.append(''.join(lemmas))

print('time = ', str(datetime.now() - start_time))

time =  0:00:05.581173


In [99]:
length = 0
for line in res_book_mystem:
  length += len(line) -2
print('Result length: ',length)

Result length:  1050114


In [100]:
res_book_mystem

['libru электронный библиотека\n',
 'название книга тихий дон книга\n',
 'автор шолохов михаил\n',
 'жанр классика\n',
 'адрес книга httpwwwlibrubookstihiydonknigiihtml\n',
 'михаил шолохов\n',
 'тихий дон\n',
 'книга\n',
 'сохамито славный землюшка наш распахать\n',
 'распахать наш землюшка лошадиный копыто\n',
 'засеивать славный землюшка казацкий голова\n',
 'украшенто наш тихий дон молодой вдова\n',
 'цвести наш батюшка тихий дон сирота\n',
 'наполнять волна тихий дон отцовский\n',
 'материнский слеза\n',
 'ой наш батюшка тихий дон\n',
 'ой тихий дон мутнехонек течь\n',
 'ах тихий дон мутна течь\n',
 'дно тихий дон студеный ключ бить\n',
 'посередь тихий дон белый рыбица мутить\n',
 'старинный казачий песня\n',
 'книга первый\n',
 'часть первый\n',
 'i\n',
 'мелеховский двор самый край хутор воротца скотиний база вести север дон крутой восьмисаживать спуск меж замшелый прозелень меловой глыба берег перламутровый россыпь ракушка серый изломистый кайма нацеловывать волна галька далек

Слов в романе: 	106356

###Токенизация

In [22]:
text = "Я учусь в физико-математическом институте. Меня зовут К.С.Астанова."

In [23]:
!pip install tensorflow-text spacy==3.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 57.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 80.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 659.5/659.5 kB 72.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 20.3 MB/s eta 0:00:00
  Attempting uninstall: wasabi
    Found existing installation: wasabi 1.1.1
    Uninstalling wasabi-1.1.1:
      Successfully uninstalled wasabi-1.1.1
  Attempting uninstall: typer
    Found existing installation: typer 0.7.0
    Uninstalling typer-0.7.0:
      Successfully uninstalled typer-0.7.0
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.7
    Uninstalling pydantic-1.10.7:
      Successfully uninstalled pydantic-1.10.7
  Attempting uninstall: thinc
    Found existing installation: thinc 8.1.9
    Uninstalling thinc-8.1.9:
      Success

In [24]:
import nltk
import spacy

KeyboardInterrupt: ignored

In [ ]:
nltk.download('punkt')

In [ ]:
from nltk.corpus.reader.tagged import word_tokenize
words = word_tokenize(text)
words

In [ ]:
for w in words:
  print(morph.parse(w)[0])

In [ ]:
!python -m spacy download ru_core_news_sm

In [ ]:
nlp = spacy.load('ru_core_news_sm')
doc = nlp(text)
tokens = []
for token in doc:
  tokens.append(str(token))
#tokens
for token in tokens:
  print(morph.parse(token)[0])